In [1]:
suppressMessages(require(ggpubr))
suppressMessages(require(tidyverse))
suppressMessages(require(data.table))
suppressMessages(require(mixtools))
suppressMessages(require(grid))
suppressMessages(require(viridis))
suppressMessages(require(RColorBrewer))
suppressMessages(require(venn))
suppressMessages(require(ggExtra))


get_density <- function(x, y, ...) {
    require(MASS)
    dens <- MASS::kde2d(x, y, ...)
    ix <- findInterval(x, dens$x)
    iy <- findInterval(y, dens$y)
    ii <- cbind(ix, iy)
    return(dens$z[ii])
}

plot_mix_comps <- function(x, mu, sigma, lam) {
    lam * dnorm(x, mu, sigma)
}
options(repr.plot.width = 4, repr.plot.height = 4)

theme_set(theme_bw(base_size = 8))

In [ ]:
sample="JYH_857_1_2"
lower.th ="100"
th.prob<-.1
log.base <- 10

In [ ]:
lower.th <- as.integer(lower.th)
# qc <- fread(paste0(sample, '.qc_metrics.txt')) %>% mutate(log_uniq_usable_reads
# = log(unique_usable_reads +
qc <- fread(paste0(sample, "/", sample, ".qc_metrics.txt")) %>% mutate(log_uniq_usable_reads = log(unique_usable_reads + 
    1, base = log.base))

## 1. Determine total_reads threshold

In [ ]:
options(repr.plot.width = 6, repr.plot.height = 6)

set.seed(1)
wait <- qc %>% filter(unique_usable_reads > lower.th) %>% pull(log_uniq_usable_reads)
suppressMessages(mixmdl <- normalmixEM(wait, k = 2))

tot_barcodes <- nrow(qc)
tot_usable_reads <- sum(qc$unique_usable_reads)
p0 <- ggplot(qc) + geom_histogram(aes(x = log_uniq_usable_reads, ..count..), binwidth = 0.05,
    alpha = 0.5, color = "black") + theme_bw() + coord_cartesian(xlim = c(0, log(10^5,base=log.base)))
p1 <- p0 + annotation_custom(grobTree(textGrob(paste0("Total barcodes", "\n", tot_barcodes,
    "\nTotal unique reads:", "\n", round(tot_usable_reads/10^6), "M"), x = 0.9, y = 0.9,
    hjust = 1, vjust = 1, gp = gpar(size = 4))))+ xlab('')

tot_cells <- sum(qc$unique_usable_reads > lower.th)
tot_reads <- sum(qc %>% filter(unique_usable_reads > lower.th) %>% pull(unique_usable_reads))
p2 <- p0 %+% (qc %>% filter(unique_usable_reads > lower.th)) + geom_vline(xintercept = log(lower.th+1,base=log.base),
    color = "black", linetype = 2) + annotation_custom(grobTree(textGrob(paste0("Total barcodes(reads>",
    lower.th, "):", "\n", tot_cells, ",", round(tot_cells/tot_barcodes *
        100), "%", "\nTotal reads:", round(tot_reads/10^6), "M,", round(tot_reads/tot_usable_reads *
        100), "%"), x = 0.05, y = 0.9, hjust = 0, vjust = 1, gp = gpar(size = 4))))+ xlab('')

threshold = qnorm(th.prob, mixmdl$mu[2], mixmdl$sigma[2])
final_cells <- sum(qc$log_uniq_usable_reads > threshold)
final_reads <- sum(qc %>% filter(log_uniq_usable_reads > threshold) %>% pull(unique_usable_reads))


p3 <- ggplot(qc %>% filter(unique_usable_reads > lower.th)) + geom_histogram(aes(x = log_uniq_usable_reads,
    ..density..), binwidth = 0.05, alpha = 0.5, color = "black") + theme_bw() + coord_cartesian(xlim = c(0,log(10^5,base=log.base)
    )) + stat_function(geom = "line", fun = plot_mix_comps, args = list(mixmdl$mu[1],
    mixmdl$sigma[1], lam = mixmdl$lambda[1]), colour = "red", lwd = 1) + stat_function(geom = "line",
    fun = plot_mix_comps, args = list(mixmdl$mu[2], mixmdl$sigma[2], lam = mixmdl$lambda[2]),
    colour = "green", lwd = 1) + geom_vline(xintercept = mixmdl$mu, color = c("red",
    "green")) + geom_vline(xintercept = threshold, color = "green", linetype = 2) +
    annotation_custom(grobTree(textGrob(paste0("log likelihood:", round(mixmdl$loglik),
        "\n", "G1.median=", round(log.base^mixmdl$mu[1] - 1), "\n", "G2.median=", round(log.base^mixmdl$mu[2] -
            1), "\n", "threshold=", round(log.base^threshold - 1), ",prob=", round(th.prob,
            2), "\n", "(", final_cells, " cells,", round(final_cells/tot_barcodes *
            100), "%)", "\n(", round(final_reads/10^6), "M reads,", round(final_reads/tot_usable_reads *
            100), "%)"), x = 0.05, y = 0.95, hjust = 0, vjust = 1, gp = gpar(size = 4))))+ xlab(paste0("Usable reads (log",log.base,")"))
threshold.total <- threshold
ggarrange(p1, p2, p3, nrow = 3)


## 2. Determine FRoP threshold 

In [ ]:
options(repr.plot.width = 4, repr.plot.height = 3)
qc.2 <- qc %>% filter(unique_usable_reads > lower.th)


p0 <- ggplot(qc.2) + geom_histogram(aes(x = frac_reads_in_promoters, 
    ..density..), binwidth = 0.005, alpha = 0.5, color = "black") + theme_bw()  #+ coord_cartesian(xlim = c(0, 0.25))
plot_mix_comps <- function(x, mu, sigma, lam) {
    lam * dnorm(x, mu, sigma)
}

mixmdl <- normalmixEM(qc.2 %>% pull(frac_reads_in_promoters), k = 2)
# threshold = qnorm(th.prob, mixmdl$mu[2], mixmdl$sigma[2])

threshold.FRoP = qnorm(0.05, mixmdl$mu[2], mixmdl$sigma[2])

p0 + stat_function(geom = "line", fun = plot_mix_comps, args = list(mixmdl$mu[1], 
    mixmdl$sigma[1], lam = mixmdl$lambda[1]), colour = "red", lwd = 1) + stat_function(geom = "line", 
    fun = plot_mix_comps, args = list(mixmdl$mu[2], mixmdl$sigma[2], lam = mixmdl$lambda[2]), 
    colour = "green", lwd = 1) + geom_vline(xintercept = threshold.FRoP, linetype = 2, 
    col = 3)


In [ ]:
plotDensity <- function(qc = qc.2) {
    qc$density <- get_density(qc$log_uniq_usable_reads, qc$frac_reads_in_promoters, 
        n = 100)
    p1 <- ggplot(qc) + geom_point(aes(log_uniq_usable_reads, frac_reads_in_promoters, 
        color = density), size = 0.5) + scale_color_viridis()
    q05 <- quantile(p1$data$density, 0.45)
    p1$data$density[p1$data$density > q05] <- q05
    p1 <- p1 + geom_vline(xintercept = threshold.total, linetype = 2) + geom_hline(yintercept = threshold.FRoP, 
        linetype = 2)
    ggMarginal(p1, type = "histogram")
}
plotDensity()

## 3. Determine FPoU threshold

In [ ]:
qc.2 <- qc %>% filter(frac_reads_in_promoters > 0 & unique_usable_reads > lower.th) %>% 
    mutate(FPoU_log10 = -log10(frac_promoters_used), FRoP_log10 = -log10(frac_reads_in_promoters))
p0 <- ggplot(qc.2) + geom_histogram(aes(x = FPoU_log10, ..density..), bins = 100, 
    alpha = 0.5, color = "black") + theme_bw()


mixmdl <- normalmixEM(qc.2 %>% pull(FPoU_log10), k = 2)
threshold.FPoU_log10 = qnorm(0.95, mixmdl$mu[1], mixmdl$sigma[1])

p0 + stat_function(geom = "line", fun = plot_mix_comps, args = list(mixmdl$mu[1], 
    mixmdl$sigma[1], lam = mixmdl$lambda[1]), colour = "red", lwd = 1) + stat_function(geom = "line", 
    fun = plot_mix_comps, args = list(mixmdl$mu[2], mixmdl$sigma[2], lam = mixmdl$lambda[2]), 
    colour = "green", lwd = 1) + geom_vline(xintercept = threshold.FPoU_log10, linetype = 2)



In [ ]:
options(repr.plot.width = 5, repr.plot.height = 3)

p0 <- ggplot(qc.2, aes(log_uniq_usable_reads, frac_reads_in_promoters))
p0 + geom_point(aes(color = FPoU_log10 <= threshold.FPoU_log10), size = 0.25, alpha = 0.25) + 
    geom_vline(xintercept = threshold.total, linetype = 2) + geom_hline(yintercept = threshold.FRoP, 
    linetype = 2)

In [ ]:
options(repr.plot.width = 4, repr.plot.height = 3)
p0 + geom_point(aes(color = FPoU_log10), size = 0.25, alpha = 0.25) + scale_color_gradientn(colours = rev(brewer.pal(9, 
    name = "Spectral")))

## 4.10x multiplets

In [ ]:
multiplets <- fread(paste0("~/data/outputs/10xATAC/", sample, "/outs/", sample, "_excluded_barcodes.csv")) %>% 
    mutate(`Excluded Barcode` = paste0(sample, "_", sub("-1", "", `Excluded Barcode`))) %>% 
    dplyr::select(-`Linked Barcode`)
multiplets %>% head(1)
multiplets %>% nrow
qc.2$isMulti = (qc.2$V1 %in% multiplets$`Excluded Barcode`)
qc.2 <- qc.2 %>% left_join(multiplets, by = c(V1 = "Excluded Barcode")) %>% mutate(`Exclusion Reason` = replace_na(`Exclusion Reason`, 
    "keep"))

options(repr.plot.width = 4, repr.plot.height = 3)

ggplot(qc.2, aes(log_uniq_usable_reads, frac_reads_in_promoters)) + geom_point(aes(color = `Exclusion Reason`), 
    size = 0.45, alpha = 0.5) + geom_vline(xintercept = threshold.total, linetype = 2) + 
    geom_hline(yintercept = threshold.FRoP, linetype = 2) + scale_color_manual(values = c("red", 
    "blue", "grey"))

In [ ]:
plotDensity(qc = qc.2 %>% filter(!isMulti))

## 5. Summary of thresholds 

In [ ]:
threshold.FPoU_log10
threshold.total
threshold.FRoP

In [ ]:
venn(list(total_pass = qc.2$V1[which(qc.2$log_uniq_usable_reads > threshold.total)], 
    FRoP_pass = qc.2$V1[which(qc.2$frac_reads_in_promoters > threshold.FRoP)], FRoU_pass = qc.2$V1[which(qc.2$FPoU_log10 < 
        threshold.FPoU_log10)], multiplets = multiplets$`Excluded Barcode`),ellipse=T)

## 6. Save thresholds

In [ ]:
fwrite(x = data.frame(min_total = round(10^threshold.total - 1), min_FRoP = threshold.FRoP, 
    min_FPoU_log10 = threshold.FPoU_log10, n_cell_passed = sum((qc.2$log_uniq_usable_reads > 
        threshold.total) & (qc.2$frac_reads_in_promoters > threshold.FRoP))), file = paste0(sample, 
    "/", sample, ".qc_thresholds.txt"))

fwrite(x = multiplets, file = paste0(sample,"/", sample, ".multiplets.txt"))